In [89]:
import os
import requests

In [90]:
r = requests.get('https://usegalaxy.org/api/tools')

In [91]:
usegalaxy_tools = r.json()

In [92]:
def get_chunks(l, size):
    for i in xrange(0, len(l), size):
        yield l[i: i + size]

In [93]:
dockerfile = """
# Galaxy - usegalaxy.org clone
#
# VERSION       0.1

FROM bgruening/galaxy-stable

MAINTAINER Bjoern A. Gruening, bjoern.gruening@gmail.com

ENV GALAXY_CONFIG_BRAND usegalaxy.org Clone


"""               

repositories = list()
for section in usegalaxy_tools:
    section_name = section.get('name', '')
    if 'elems' not in section:
        continue
    for tool in section['elems']:
        idd = tool['id']
        if 'toolshed' in idd:
            items = idd.split('/')
            ts = items[0]
            owner = items[2]
            name = items[3]
            repo = "    --url https://%s -o %s --name %s --panel-section-name '%s'" % (ts, owner, name, section_name)
            repositories.append(repo)

for repos in get_chunks(list(set(repositories)), 10):
    dockerfile += "RUN install-repository \\\n"
    dockerfile += " \\\n".join(repos)
    dockerfile += "\n\n"
with open(os.path.join(os.path.expanduser('~'), 'usegalaxy.Dockerfile'), 'w+') as handle:
    handle.write(dockerfile)
print(dockerfile)


# Galaxy - usegalaxy.org clone
#
# VERSION       0.1

FROM bgruening/galaxy-stable

MAINTAINER Bjoern A. Gruening, bjoern.gruening@gmail.com

ENV GALAXY_CONFIG_BRAND usegalaxy.org Clone


RUN install-repository \
    --url https://toolshed.g2.bx.psu.edu -o devteam --name tabular_to_fasta --panel-section-name 'FASTA manipulation' \
    --url https://toolshed.g2.bx.psu.edu -o devteam --name gi2taxonomy --panel-section-name 'Metagenomic analyses' \
    --url https://toolshed.g2.bx.psu.edu -o devteam --name lca_wrapper --panel-section-name 'Metagenomic analyses' \
    --url https://toolshed.g2.bx.psu.edu -o devteam --name samtools_calmd --panel-section-name 'NGS: SAMtools' \
    --url https://toolshed.g2.bx.psu.edu -o devteam --name intersect --panel-section-name 'Operate on Genomic Intervals' \
    --url https://toolshed.g2.bx.psu.edu -o devteam --name fasta_nucleotide_changer --panel-section-name 'FASTA manipulation' \
    --url https://toolshed.g2.bx.psu.edu -o devteam --name xy_plot -